# 课时45 变量与自动微分运算

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
sb.set_style('darkgrid')
# pathlib相比os.path更好用
import pathlib
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.4.0


# 1. Variable变量

In [4]:
# 需要注意的是tf.Variable得到的变量是整个模型中可以进行更改更新的参数
v = tf.Variable(0.0)
(v + 1).numpy()

1.0

In [5]:
# 下面展示如何直接修改变量的值
v.assign(5)
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [6]:
v.assign_add(1)
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [7]:
v.read_value()

<tf.Tensor: id=32, shape=(), dtype=float32, numpy=6.0>

# 2. 自动微分运算

In [4]:
# tape 磁带，会自动的跟踪记录一个变量的计算过程，然后方便进行微分计算
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
# t.gradient(target=loss, sources=w)代表的是目标target对变量w的微分
grad = tape.gradient(target=loss, sources=w)
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [6]:
# 如果是一个常量的话，要计算其微分，需要给定tf.GradientTape()一些要求以便记录下运算过程
w2 = tf.constant([[3.0]])
with tf.GradientTape() as t:
    # watch用于记录常量的运算过程
    t.watch(w2)
    loss2 = w2 * w2
# t.gradient(target=loss, sources=w)代表的是目标target对变量w的微分
grad2 = t.gradient(target=loss2, sources=w2)
grad2

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

需要注意两点：
>1. 自动微分运算对于变量(Variable)和常量(Constant)的跟踪，它都要求这两个是float数据类型, 否则是求解不出梯度的；
>2. 对于tf.GradientTape()所持有的资源(也就是记录的这些运算)，在我们调用了t.gradient()方法之后会立即释放这些资源，这个意思就是说在同一次计算中如果我们要计算多个微分的话，这个时候是不行的，因为t计算一次就会将资源释放，如果要计算多个变量多次微分的话，这个时候需要在tf.GradientTape()中添加一个参数，也就是persistent=True，如下：

In [11]:
# 如果是一个常量的话，要计算其微分，需要给定tf.GradientTape()一些要求以便记录下运算过程
w3 = tf.constant([[3.0]])
# persistent=True会让tf.GradientTape()持久，而不是t.gradient()运行完之后就释放资源
with tf.GradientTape(persistent=True) as t:
    # watch用于记录常量的运算过程
    t.watch(w3)
    y = w3 * w3
    z = y * y
# t.gradient(target=loss, sources=w)代表的是目标target对变量w的微分
dy_dw = t.gradient(target=y, sources=w3)
dz_dw = t.gradient(target=z, sources=w3)
# 这里直接调用的话会爆RuntimeError，因为在第一个t.gradient()运行完之后资源已经释放了
# dy_dw
dz_dw

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[108.]], dtype=float32)>

# 3. 使用tf.GradientTape实现梯度下降以及自定义的训练与循环

In [73]:
# 加载MNIST数据集
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# 以tf.Dataset形式加载数据(对于MNIST数据集，它是没有第三个维度Channel的，这里添加上它的第三个维度)
train_images = tf.expand_dims(train_images, -1)
# 由于tf.GradientTape()要求的数据类型是float，因此这里需要改变MNIST数据集图片的数据类型
# 并在转换数据类型的过程中对图片数据进行归一化
train_images = tf.cast(train_images/255, tf.float32)
# 对于MNIST数据集的标签，数据类型是uint8(无符号8位)，这里为了方便计算，将其转换为int64类型
train_labels = tf.cast(train_labels, tf.int64)
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

test_images = tf.expand_dims(test_images, -1)
test_images = tf.cast(test_images/255, tf.float32)
test_labels = tf.cast(test_labels, tf.int64)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [74]:
# 对数据进行打乱以及batch划分
dataset = dataset.shuffle(buffer_size=10000).batch(batch_size=32)
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [75]:
test_dataset = test_dataset.batch(batch_size=32)
test_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [76]:
# 建立模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=[3, 3], activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=32, kernel_size=[3, 3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=10, activation='softmax')])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
global_average_pooling2d_2 ( (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 5,130
Trainable params: 5,130
Non-trainable params: 0
_________________________________________________________________


In [78]:
# 进行自定义训练
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# 由于dataset是tf.Dataset对象，因此在eager模型下是直接可迭代的(这里设定了batch=32，因此每次迭代都取出32个数据)：
features, labels = next(iter(dataset))

In [79]:
# model内置call方法，因此可以直接调用
predictions = model(features)
# 还没进行训练的时候预测的进度不好
tf.argmax(predictions, axis=1)

<tf.Tensor: id=1305823, shape=(32,), dtype=int64, numpy=
array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=int64)>

In [80]:
# 定义损失函数，每次传入数据之后都会返回相应的损失值
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [81]:
# 定义两个tf.keras.metrics对象用于计算自定义训练中的loss均值和正确率
train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [82]:
# 计算一步对应的损失函数与模型参数的梯度
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        # loss_step = loss(model, images, labels)
        # 为了方便tf.keras.metrics模块的演示，将上面这个代码拆成两步，不再调用loss计算函数
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    # apply_gradients()代表将优化方法应用到参数的更新中
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    # 这里是新增的部分，每一步都计算均值和正确率
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [83]:
# 计算一步对应的损失函数与模型参数的梯度
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [84]:
def train_model():
    # 训练3个epochs
    for epoch in range(3):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch %i loss is %f, acc is %f'%(epoch, 
                                                train_loss.result(),
                                                train_accuracy.result()))
        # 每个batch结束就重置这一轮的train_loss和train_accuracy
        train_loss.reset_states()
        train_accuracy.reset_states()
        
        # ------------------------------------------------------------
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch %i test_loss is %f, test_acc is %f'%(epoch, 
                                                test_loss.result(),
                                                test_accuracy.result()))
        
        # 每个batch结束就重置这一轮的train_loss和train_accuracy
        test_loss.reset_states()
        test_accuracy.reset_states()

In [86]:
train_model()

# 4. tf.keras.metrics汇总计算模块
tf.keras.metrics这个模型用于计算模型训练过程中的各个数据情况
## 4.1 tf.keras.metrics模块计算均值

In [13]:
# tf.keras.metrics.Mean用于计算均值，这里是计算准确率的均值
m = tf.keras.metrics.Mean('acc')
# 如果需要计算一些数值的均值，可以是一个一个的传入到m(value)中，最后再使用m.result()
# m(10)
# m(20)
# m.result()
# 或者直接给一个列表进行计算
m([10, 20, 30, 50])
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=27.5>

In [48]:
# 用于重置状态(避免记录上面的结果影响后面的计算)
m.reset_states()

## 4.2 tf.keras.metrics计算准确率

In [53]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')
# a(y, y_hat)
a(labels, model(features))
a.result()

<tf.Tensor: id=270449, shape=(), dtype=float32, numpy=0.09375>